In [1]:
# Loading data and label
import numpy as np
emotion_data = np.load('/sda/rina_1921cs13/Fake_Emo/UnBiasedEmo/data.npy')
emotion_label = np.load('/sda/rina_1921cs13/Fake_Emo/UnBiasedEmo/label.npy')
print('Emotion data shape', emotion_data.shape)
print('Emotion label shape', emotion_label.shape)
label_keys = {}
item_lab = ["anger", "fear", "joy", "love", "sadness", "surprise"]
for i, emotion in enumerate(item_lab):
    label_keys[i] = emotion
print(label_keys)

Emotion data shape (3045, 224, 224, 3)
Emotion label shape (3045,)
{0: 'anger', 1: 'fear', 2: 'joy', 3: 'love', 4: 'sadness', 5: 'surprise'}


In [3]:
# Create two labels - real(1), fake(0) (Multilabel classification)
from collections import Counter
combine_label = []
for label in emotion_label:
    # fear, surprise, anger - false emo
    if label == 0 or label == 1 or label == 5:
        combine_label.append(0)
    elif label == 2 or label == 3 or label == 4:
        combine_label.append(1)
print('Fake-Real distribution', Counter(combine_label))
combine_label = np.array(combine_label)
print('Combine label shape', combine_label.shape)

Fake-Real distribution Counter({1: 1995, 0: 1050})
Combine label shape (3045,)


In [4]:
#TODO: Fine-Tune on Imagenet-pre-trained ResNet-50 model
import os
import sys
import numpy as np
from numpy import asarray,zeros
import pandas as pd 
from sklearn.model_selection import train_test_split
import torch
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler, Dataset
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import matplotlib.pyplot as plt
import copy
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import classification_report
import transformers
from transformers import BertTokenizer
from transformers import AdamW, get_linear_schedule_with_warmup
from transformers import AutoTokenizer, AutoModel, AutoConfig, AdamW, get_linear_schedule_with_warmup
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import timm

In [5]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    os.environ['CUDA_ENVIRONMENT_DEVICES'] = "0"
else:
    device = torch.device("cpu")
print("Using device", device)

Using device cuda


In [6]:
num_images, width, height, num_channels = emotion_data.shape
emotion_data = np.reshape(emotion_data, newshape=(num_images, num_channels, width, height))
print('Emotion data shape', emotion_data.shape)

Emotion data shape (3045, 3, 224, 224)


In [7]:
# Convert to tensor
data_tensor = torch.tensor(emotion_data, device=device)
labels_tensor = torch.tensor(combine_label, dtype=torch.long, device=device)
print('data tensor shape', data_tensor.shape)
print('Labels tensor shape', labels_tensor.shape)

data tensor shape torch.Size([3045, 3, 224, 224])
Labels tensor shape torch.Size([3045])


In [8]:
# NOTE: Splitting data into train and test
train_data, test_data, train_labels, test_labels = train_test_split(data_tensor, labels_tensor, test_size=0.2, random_state=43)

In [9]:
#TODO: Add Pytorch DataLoader
def get_data_loader(batch_size, data, labels, split_type='train'):
	data = TensorDataset(data, labels)
	if split_type == 'train':
		sampler = RandomSampler(data)
		dataloader = DataLoader(data, sampler=sampler, batch_size=batch_size)
	elif split_type == 'val':
		sampler = SequentialSampler(data)
		dataloader = DataLoader(data, sampler=sampler, batch_size=batch_size)
	return data, sampler, dataloader

In [10]:
# Getting the dataloader
batch_size = 128
train_data, train_sampler, train_dataloader = get_data_loader(batch_size, train_data, train_labels, 'train')
test_data, test_sampler, test_dataloader = get_data_loader(batch_size, test_data, test_labels, 'val')

In [9]:
# # TODO: Test the resnet-50 model
# res50model = timm.create_model('resnet50', pretrained=True).to(device)
# res50base = ResNetBottom(res50model).to(device) # Output-2048 dim
# # Forward pass on the data
# output = res50base(data_tensor[0:5,:,:,:])
# print(output.shape)

In [11]:
#TODO: Define Resent-50 model
class ResNetBottom(nn.Module):
    def __init__(self, original_model):
        super(ResNetBottom, self).__init__()
        self.features = nn.Sequential(*list(original_model.children())[:-1])
        
    def forward(self, x):
        x = self.features(x)
        return x

class ResnetBased(nn.Module):
    def __init__(self):
        super(ResnetBased, self).__init__()
        self.vision_base_model = timm.create_model('resnet50', pretrained=True)
        self.vision_model_head = ResNetBottom(self.vision_base_model)
        self.project_1 = nn.Linear(2048, 1024, bias=True)
        self.project_2 = nn.Linear(1024, 512, bias=True)
        self.project_3 = nn.Linear(512, 128, bias=True)
        self.tanh1 = nn.Tanh()
        self.tanh2 = nn.Tanh()
        self.tanh3 = nn.Tanh()
        self.drop1 = nn.Dropout()
        self.drop2 = nn.Dropout()
        self.drop3 = nn.Dropout()
        self.classification = nn.Linear(128, 6, bias=True)
    def forward(self, img_features):
        with torch.no_grad():
            img_out = self.vision_model_head(img_features)
        emotion_features = self.tanh3(self.project_3(self.tanh2(self.project_2(self.tanh1(self.project_1(img_out))))))
        class_out = self.classification(emotion_features)
        return class_out

In [12]:
# Optimizer and scheduler
def get_optimizer_scheduler(name, model, train_dataloader_len, epochs, lr_set):
	if name == "Adam":
		optimizer = AdamW(model.parameters(),
                  lr = lr_set, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
		)
	total_steps = train_dataloader_len * epochs

	# Create the learning rate scheduler.
	scheduler = get_linear_schedule_with_warmup(optimizer, 
												num_warmup_steps = total_steps//2, # Default value in run_glue.py
												num_training_steps = total_steps)
	return optimizer, scheduler

In [13]:
# Get the model
emo_model = ResnetBased().to(device)
# Getting the optimizer and scheduler
epochs = 100
lr = 3e-5 # Less LR
iters_to_accumulate = 2
name = "Adam"
criterion = nn.CrossEntropyLoss()
optimizer, scheduler = get_optimizer_scheduler(name, emo_model, len(train_dataloader), epochs, lr)

In [14]:
################ Evaluating Loss ######################
#######################################################
def evaluate_loss(net, device, criterion, dataloader):
    net.eval()
    mean_loss = 0
    count = 0
    with torch.no_grad():
        for it, (input_data, labels) in enumerate(tqdm(dataloader)):
            input_data, labels = input_data.to(device), labels.to(device)
            logits = net(input_data)
            mean_loss += criterion(logits.squeeze(-1), labels).item() # initially it was logits.squeeze(-1)
            count += 1
    return mean_loss / count

In [15]:
################ Flat Accuracy Calculation ####################
###############################################################
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)
################ Validation Accuracy Calculation ####################
###############################################################
def evaluate_accuracy(model, device, validation_dataloader):
	model.eval()
	# Tracking variables 
	eval_loss, eval_accuracy = 0, 0
	nb_eval_steps, nb_eval_examples = 0, 0
	# Evaluate data for one epoch
	for batch in validation_dataloader:
	    # Add batch to GPU
	    batch = tuple(t.to(device) for t in batch)	    
	    # Unpack the inputs from our dataloader
	    b_inputs, b_labels = batch	    
	    
	    # Telling the model not to compute or store gradients, saving memory and
	    # speeding up validation
	    with torch.no_grad(): 
	    	# Forward pass, calculate logit predictions.
	        # This will return the logits rather than the loss because we have
	        # not provided labels.
	    	logits = model(b_inputs)       

	    # Move logits and labels to CPU
	    logits = logits.detach().cpu().numpy()
	    label_ids = b_labels.to('cpu').numpy()
	    
	    # Calculate the accuracy for this batch of test sentences.
	    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
	    
	    # Accumulate the total accuracy.
	    eval_accuracy += tmp_eval_accuracy

	    # Track the number of batches
	    nb_eval_steps += 1
	accuracy = eval_accuracy/nb_eval_steps
	return accuracy

In [16]:
def train_model(net, criterion, opti, lr, lr_scheduler, train_loader, val_loader, epochs, iters_to_accumulate):
    best_loss = np.Inf
    best_ep = 1
    nb_iterations = len(train_loader)
    print_every = nb_iterations // 5  # print the training loss 5 times per epoch
    iters = []
    train_losses = []
    val_losses = []
    # Iterating over all epochs
    for ep in range(epochs):
        net.train()
        running_loss = 0.0
        for it, (data_inputs, labels) in enumerate(tqdm(train_loader)):

            # Converting to cuda tensors
            data_inputs, labels = data_inputs.to(device), labels.to(device)
    		
            # Obtaining the logits from the model
            logits = net(data_inputs)
            # print(logits.device)

            # Computing loss
            # print(logits.squeeze(-1).shape)
            # print(labels.shape)
            loss = criterion(logits.squeeze(-1), labels)
            loss = loss / iters_to_accumulate  # Normalize the loss because it is averaged

            # Backpropagating the gradients
            # Calls backward()
            loss.backward()

            if (it + 1) % iters_to_accumulate == 0:
                # Optimization step
                # scaler.step() first unscales the gradients of the optimizer's assigned params.
                # If these gradients do not contain infs or NaNs, opti.step() is then called,
                # otherwise, opti.step() is skipped.
                opti.step()
                # Adjust the learning rate based on the number of iterations.
                lr_scheduler.step()
                # Clear gradients
                net.zero_grad()


            running_loss += loss.item()

            if (it + 1) % print_every == 0:  # Print training loss information
                print()
                print("Iteration {}/{} of epoch {} complete. Loss : {} "
                      .format(it+1, nb_iterations, ep+1, running_loss / print_every))

                running_loss = 0.0


        val_loss = evaluate_loss(net, device, criterion, val_loader)  # Compute validation loss
        val_accuracy = evaluate_accuracy(net, device, val_loader)
        print()
        print("Epoch {} complete! Validation Loss : {}".format(ep+1, val_loss))
        print("Epoch {} complete! Validation Accuracy : {}".format(ep+1, val_accuracy))

        if val_loss < best_loss:
            print("Best validation loss improved from {} to {}".format(best_loss, val_loss))
            print()
            net_copy = copy.deepcopy(net)  # save a copy of the model
            best_loss = val_loss
            best_ep = ep + 1

    # Saving the model
    model_name = 'emo_combine_res50'
    path_to_model='saved_models/{}_lr_{}_val_loss_{}_ep_{}.pt'.format(model_name, lr, round(best_loss, 5), best_ep)
    torch.save(net_copy.state_dict(), path_to_model)
    net.load_state_dict(torch.load(path_to_model)) # Re-Loading the best model
    print("The model has been saved in {}".format(path_to_model))

    del loss
    torch.cuda.empty_cache()
    return net

In [17]:
# Train the model
model = train_model(emo_model, criterion, optimizer, lr, scheduler, train_dataloader, test_dataloader, epochs, iters_to_accumulate)

 25%|██▌       | 5/20 [00:00<00:02,  6.12it/s]


Iteration 4/20 of epoch 1 complete. Loss : 0.8973053842782974 


 45%|████▌     | 9/20 [00:01<00:01,  6.53it/s]


Iteration 8/20 of epoch 1 complete. Loss : 0.8974790275096893 


 65%|██████▌   | 13/20 [00:02<00:01,  6.64it/s]

...


Iteration 16/20 of epoch 100 complete. Loss : 0.2908206582069397 


 20%|██        | 1/5 [00:00<00:00,  6.34it/s]


Iteration 20/20 of epoch 100 complete. Loss : 0.2777697518467903 


100%|██████████| 5/5 [00:00<00:00,  6.53it/s]



Epoch 100 complete! Validation Loss : 0.6095451354980469
Epoch 100 complete! Validation Accuracy : 0.7006443298969073
The model has been saved in saved_models/emo_res50_lr_3e-05_val_loss_0.60696_ep_98.pt


In [18]:
def evaluate(prediction_dataloader, model, path_to_model, load = False):
	# Prediction on test set
	if load:
		print("Loading the weights of the model...")
		model.load_state_dict(torch.load(path_to_model))

	print('Evaluating on the testset')

	# Put model in evaluation mode
	model.eval()

	# Tracking variables 
	predictions , true_labels = [], []

	# Predict 
	for batch in prediction_dataloader:
	  # Add batch to GPU
	  batch = tuple(t.to(device) for t in batch)
	  
	  # Unpack the inputs from our dataloader
	  b_inputs, b_labels = batch
	  
	  # Telling the model not to compute or store gradients, saving memory and 
	  # speeding up prediction
	  with torch.no_grad():
	      # Forward pass, calculate logit predictions
	      logits = model(b_inputs)

	  # Move logits and labels to CPU
	  logits = logits.detach().cpu().numpy()
	  label_ids = b_labels.to('cpu').numpy()

	  pred_flat = np.argmax(logits, axis=1).flatten()
	  labels_flat = label_ids.flatten()
	  
	  # Store predictions and true labels
	  predictions.extend(pred_flat)
	  true_labels.extend(labels_flat)

	print('DONE.')

	# Code for result display
	print('Unbiased Emotion Classification accuracy is')
	print(metrics.accuracy_score(true_labels, predictions)*100)
	print(classification_report(true_labels, predictions, target_names = ['fake_emo', 'real_emo']))
	# Converting to csv
	# Removed transpose - check if actually required
	model_name = 'combine_emo'
	clsf_report = pd.DataFrame(classification_report(y_true = true_labels, y_pred = predictions, output_dict=True, target_names = ['fake_emo', 'real_emo']))
	clsf_report.to_csv(str('saved_models/'+model_name+'.csv'), index= True)

In [19]:
path_to_model = 'saved_models/emo_res50_lr_3e-05_val_loss_1.48489_ep_76.pt'
evaluate(test_dataloader, model, path_to_model = path_to_model, load = False)

Evaluating on the testset
DONE.
Unbiased Emotion Classification accuracy is
69.29392446633827
              precision    recall  f1-score   support

    fake_emo       0.61      0.14      0.22       197
    real_emo       0.70      0.96      0.81       412

    accuracy                           0.69       609
   macro avg       0.66      0.55      0.52       609
weighted avg       0.67      0.69      0.62       609

